In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

D:\LangChain\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview",temperature=0)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [3]:
class EvaluationSchema(BaseModel):
    feedback:str = Field(description="Detailed feedback for the essay")
    score:int = Field(description="Score out 10",ge=0,le=10)

In [4]:
structured_llm = llm.with_structured_output(EvaluationSchema)

In [11]:
class EssayState(TypedDict):
    essay:str
    language_feedback:str
    analysis_feedback:str
    clarity_feedback:str
    overall_feedback:str
    individual_scores:Annotated[list[int],operator.add]
    avg_score:float


In [12]:
def evaluate_language(state:EssayState):
    prompt = f'Evaluate the language quality of the essay and provide a feedback out of 10. \n: {state["essay"]}'
    output = structured_llm.invoke(prompt)
    return {'language_feedback':output.feedback,'individual_scores':state['individual_scores']+[output.score]}


In [13]:
def evaluate_analysis(state:EssayState):
    prompt = f'Evaluate the depth of analysis of the essay and provide a feedback out of 10. \n: {state["essay"]}'
    output = structured_llm.invoke(prompt)
    return {'analysis_feedback':output.feedback,'individual_scores':state['individual_scores']+[output.score]}

In [14]:
def evaluate_thought(state:EssayState):
    prompt = f'Evaluate the clarity of thought of the essay and provide a feedback out of 10. \n: {state["essay"]}'
    output = structured_llm.invoke(prompt)
    return {'clarity_feedback':output.feedback,'individual_scores':state['individual_scores']+[output.score]}

In [15]:
def final_evaluation(state:EssayState):
    prompt = f"Create an overall feedback for the essay based on the following feedback: \n {state['language_feedback']}, {state['analysis_feedback']}, {state['clarity_feedback']}"
    overall_feedback = llm.invoke(prompt).content
    avg = sum(state['individual_scores'])/len(state['individual_scores'])
    return {'overall_feedback':overall_feedback,'avg_score':avg}

In [16]:
graph = StateGraph(EssayState)

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

#add edges
graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')

graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')

graph.add_edge('final_evaluation',END)

workflow = graph.compile()

In [18]:
essay = "This is a sample essay."

In [21]:
initial_state = {'essay':essay}
final_state = workflow.invoke(initial_state)
print(final_state)

{'essay': 'This is a sample essay.', 'language_feedback': 'The text provided is a single, simple sentence rather than a complete essay. It lacks depth, structural complexity, and the necessary components to evaluate language proficiency such as vocabulary range, grammatical variety, or coherence. As it does not meet the requirements of an essay, it cannot be effectively assessed.', 'analysis_feedback': 'The provided text consists of a single sentence and does not contain any substantive content or analysis. As a result, it is impossible to evaluate the depth of analysis, as no arguments or supporting evidence are presented.', 'clarity_feedback': 'The input provided is a single sentence and does not constitute a complete essay. While the sentence itself is clear in its literal meaning, it lacks the depth, structure, and logical progression necessary to evaluate clarity of thought in an academic or creative context. To improve, provide a full-length essay that includes an introduction, b